In [37]:
import pandas as pd
import numpy as np
import requests

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import sys

from io import BytesIO

### 1) Загрузите данные из Google Sheets, которые находятся в (table - Лист1) при помощи библиотеки requests.
Посчитайте процент мужчин среди всех посетителей сайта, округлите ответ до целых.

In [11]:
your_link = 'https://docs.google.com/spreadsheets/d/165QBa-1tEwBF8XDblj0KPSGL6K4P3mOQQN4AU-9iqwY/gviz/tq?tqx=out:csv'

In [12]:
req = requests.get(your_link)
data = req.content

df = pd.read_csv(BytesIO(data))

In [13]:
df

,"date,gender,users"
0,"2020-06-14,Not specified,150"
1,"2020-06-14,male,117"
2,"2020-06-14,female,21"


In [17]:
# Разделяем колонку 'date,gender,users' на отдельные колонки
df[['date', 'gender', 'users']] = df['date,gender,users'].str.split(',', expand=True)

# Преобразуем колонку 'date' в тип данных datetime
df['date'] = pd.to_datetime(df['date'])

# Преобразуем колонку 'users' в числовой тип данных
df['users'] = df['users'].astype(int)

# Удаляем старую колонку, теперь когда данные в ней распределены по новым колонкам
df = df.drop('date,gender,users', axis=1)
df

,date,gender,users
0,2020-06-14,Not specified,150
1,2020-06-14,male,117
2,2020-06-14,female,21


In [19]:
text = req.text.splitlines()

In [20]:
text

['"date,gender,users"',
 '"2020-06-14,Not specified,150"',
 '"2020-06-14,male,117"',
 '"2020-06-14,female,21"']

In [21]:
header = text[0].split(',')
header

['"date', 'gender', 'users"']

In [23]:
content = [row.split(',') for row in text[1:]]
content

[['"2020-06-14', 'Not specified', '150"'],
 ['"2020-06-14', 'male', '117"'],
 ['"2020-06-14', 'female', '21"']]

In [24]:
df2 = pd.DataFrame(content, columns=header)
df2

,"""date",gender,"users"""
0,"""2020-06-14",Not specified,"150"""
1,"""2020-06-14",male,"117"""
2,"""2020-06-14",female,"21"""


In [28]:
# Уберем кавычки из заголовков столбцов
df2.columns = df2.columns.str.strip('"')

# Уберем кавычки во всех текстовых столбцах DataFrame, используя метод .map() для каждой колонки отдельно.
for column in df2.columns:
    # Применяем .map() только к колонкам с текстовыми данными
    if pd.api.types.is_string_dtype(df2[column]):
        df2[column] = df2[column].map(lambda x: x.strip('"'))

df2

,date,gender,users
0,2020-06-14,Not specified,150
1,2020-06-14,male,117
2,2020-06-14,female,21


In [33]:
df.users\
    .div(df.users.sum())\
    .mul(100)\
    .round(0)

0    52.0
1    41.0
2     7.0
Name: users, dtype: float64

### 2) При помощи функции np.where создайте серию
в которую будет записываться 1, если посещения были сделаны роботами с браузера, содержащего Chrome в названии
и 0, в других случаях.
Поместите получившуюся серию в переменную chrome_robots

In [40]:
ya_metrica = pd.read_csv('https://stepik.org/media/attachments/lesson/361619/karpov_courses_test.csv', parse_dates=['date'])
ya_metrica

,date,browser,user_type,users
0,2020-06-14,Google Chrome,People,139
1,2020-06-14,Chrome Mobile,People,35
2,2020-06-14,Google Chrome,Robots,28
3,2020-06-14,Yandex Browser,People,22
4,2020-06-14,Firefox,People,15
5,2020-06-14,Safari,People,13
6,2020-06-14,Firefox,Robots,8
7,2020-06-14,Android Browser,People,8
8,2020-06-14,Opera,People,5
9,2020-06-14,Mobile Safari,People,5


In [41]:
ya_metrica.user_type == "Robots"

0     False
1     False
2      True
3     False
4     False
5     False
6      True
7     False
8     False
9     False
10    False
11     True
12     True
13     True
14     True
15    False
16    False
17     True
18     True
19     True
20    False
21     True
22    False
23    False
Name: user_type, dtype: bool

In [42]:
ya_metrica.browser.str.contains('Chrome')

0      True
1      True
2      True
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19     True
20    False
21    False
22    False
23    False
Name: browser, dtype: bool

In [44]:
chrome_robots = np.where((ya_metrica.user_type == "Robots") & (ya_metrica.browser.str.contains('Chrome')), 1, 0)
chrome_robots = pd.Series(chrome_robots)
chrome_robots

0     0
1     0
2     1
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    1
20    0
21    0
22    0
23    0
dtype: int32

In [45]:
correspondence = {
    'People': 'Humans',
    'Robots': 'Bots'
}

In [46]:
ya_metrica.user_type.map(correspondence)

0     Humans
1     Humans
2       Bots
3     Humans
4     Humans
5     Humans
6       Bots
7     Humans
8     Humans
9     Humans
10    Humans
11      Bots
12      Bots
13      Bots
14      Bots
15    Humans
16    Humans
17      Bots
18      Bots
19      Bots
20    Humans
21      Bots
22    Humans
23    Humans
Name: user_type, dtype: object

3) У вас есть датасет sample_ads.csv с данными о кликах и показах по объявлению, в этом датасете есть пропуски.
Вычислите количество строк без пропусков в колонке click_type.
Запишите ответ в переменную not_nan_count

    ad_id - id объявления
    user_id - id пользователя
    click_type - тип клика (клик по заголовку объявления, клик по центру)

not_nan_count = df.click_type.notna().sum()

4) ⭐️Задание со звёздочкой!⭐️
В списке lst находятся различные элементы. Создайте списк new_list и сделайте следующее:
- при помощи конструкции try except и цикла попробуйте преобразовать каждое значение в листе в int
- если это получилось - добавьте преобразованное значение  в список new_list
for element in lst:
	new_list.append(int(element))

5) ⭐️Задание со звёздочкой!⭐️
А теперь попробуем воспользоваться знаниями использования Telegram API. Отправьте себе в личку любое послание, а в ответ к этому шагу напишите ник бота, от имени которого будете отправлять сообщения.